# Multiple scattering
## c)

The resulting distances deviate slightly from the exponential distribution because the exponentially distributed scattering length is not used in the case for when the total distance travelled and the scattering length combined would exceed the perviously determined absorption length. This results in the distances not being distributed exponentially.

## a) and b)

In [ ]:
def propagate(self, random_state, **kwargs):
        """Propagate the particle.

        This method propagates the particle and creates energy losses.
        The energy losses can be passed on to a Detector instance to generate
        an event.

        The first energy deposition should consist of the vertex of the
        particle, e.g. it should be:
            (self.x, self.y, 0., self.direction)
        The last energy deposition should be the point of absorption and the
        deposited energy should be equal to the particle's energy.

        Parameters
        ----------
        random_state : RNG object
            The random number generator to use.
        **kwargs
            Additional keyword arguments.

        Returns
        -------
        array_like
            The list of energy losses. Each energy loss consists of a tuple of
            [x-position, y-position, deposited Energy, direction].
            Shape: [num_losses, 4]
        """

        # ---------
        # Exercise:
        # ---------
        #   MC Multiple Scattering (exercises/mc_multiple_scattering.py)
        
        # drawing absorption length at which the partical is absorbed
        absorption_length = random_state.exponential(self.absorption_length)

        # initializing variable for travelled distance, helps keep tracking for 
        # when particle is absorbed
        distance_travelled = 0

        # first scattering process without energy loss
        energy_depositions = [(self.x, self.y, 0., self.direction)]

        # implementing loop that runs until particle is absorbed
        while distance_travelled < absorption_length:
            
            # distance until scattering occurs
            scattering_length = random_state.exponential(self.scattering_length)


            # case: travel distance will not exceed the absorption length
            if distance_travelled + scattering_length < absorption_length:
                # updating position of particle
                self.x = self.x + scattering_length * np.cos(self.direction)
                self.y = self.y + scattering_length * np.sin(self.direction)

                # setting energy loss at point of scattering (in this case E_dep = 0) 
                E_dep = 0.

                # appending parameters to energy_depositions array
                energy_depositions.extend([(self.x, self.y, E_dep, self.direction)])

                # drawing new direction of particle using rvs method
                self.direction = self.direction + self.angle_distribution.rvs(random_state)

                # updating travelled distance
                distance_travelled = distance_travelled + scattering_length


            # case: travel distance would exceed the absorption length, therefore calculating
            # the distance until the absorption length is reached    
            else:
                distance_end = absorption_length - distance_travelled

                # updating position of particle
                self.x = self.x + distance_end * np.cos(self.direction)
                self.y = self.y + distance_end * np.sin(self.direction)

                # setting energy loss at point of scattering 
                E_dep = self.energy
                self.energy = 0

                # appending parameters to energy_depositions array
                energy_depositions.extend([(self.x, self.y, E_dep, self.direction)])

                # drawing new direction of particle using rvs method
                self.direction = self.direction + self.angle_distribution.rvs(random_state)
                
                # updating travelled distance
                distance_travelled = distance_travelled + distance_end


        return energy_depositions